# Setup

## python version

In [1]:
from platform import python_version
print(python_version())

3.7.9


## Librerie

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import mechanicalsoup as ms
import scrapy
import selenium
import lxml
import re
import requests
import pandas as pd
from pandas import compat
import json
from requests import get
import pprint
#import html5lib

## Setup Selenium

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 1. `pokemondb` Tables 

## POKEDEX: Pokemon list with stats (id, nome, tipo, total, hp, attack, defense, sp. atk., sp. def, speed)

Url della risorsa, HTTPResponse Object e BeautifulSoupObject della pagina

In [37]:
pokedex_URL = "https://pokemondb.net/pokedex/all"
pokedex_page = requests.get(pokedex_URL)

In [38]:
pokedex_soup = BeautifulSoup(pokedex_page.content, features="html.parser")

In [39]:
pokedex_table = pokedex_soup.find(id="pokedex")

In [40]:
pokedex_df = pd.read_html(str(pokedex_table))[0]

In [41]:
pokedex_name_split = pokedex_df['Name'].str.split(n = 1, expand = True)
pokedex_df['Name'] = pokedex_name_split[0]
pokedex_df.insert(2, "Variant", pokedex_name_split[1])
#pokedex_df['Type'] = pokedex_df['Type'].replace(' ', ', ', regex = True)

In [42]:
pokedex_type_split = pokedex_df['Type'].str.split(n = 1, expand = True)
pokedex_df.drop('Type', inplace = True, axis = 1)
pokedex_df.insert(2, "Type1", pokedex_type_split[0])
pokedex_df.insert(3, "Type2", pokedex_type_split[1])

In [43]:
pokedex_df

,#,Name,Type1,Type2,Variant,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,None,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,None,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,None,525,80,82,83,100,100,80
3,3,Venusaur,Grass,Poison,Mega Venusaur,625,80,100,123,122,120,80
4,4,Charmander,Fire,None,None,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...
1070,902,Basculegion,Water,Ghost,Female,530,120,92,65,100,75,78
1071,903,Sneasler,Poison,Fighting,None,510,80,130,60,40,80,120
1072,904,Overqwil,Dark,Poison,None,510,85,115,95,65,65,85
1073,905,Enamorus,Fairy,Flying,Incarnate Forme,580,74,115,70,135,80,106


In [53]:
pokedex_json = json.dumps(list(pokedex_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4)

In [55]:
print(pokedex_json)

[
    {
        "#": 1,
        "Name": "Bulbasaur",
        "Type1": "Grass",
        "Type2": "Poison",
        "Total": 318,
        "HP": 45,
        "Attack": 49,
        "Defense": 49,
        "Sp. Atk": 65,
        "Sp. Def": 65,
        "Speed": 45
    },
    {
        "#": 2,
        "Name": "Ivysaur",
        "Type1": "Grass",
        "Type2": "Poison",
        "Total": 405,
        "HP": 60,
        "Attack": 62,
        "Defense": 63,
        "Sp. Atk": 80,
        "Sp. Def": 80,
        "Speed": 60
    },
    {
        "#": 3,
        "Name": "Venusaur",
        "Type1": "Grass",
        "Type2": "Poison",
        "Total": 525,
        "HP": 80,
        "Attack": 82,
        "Defense": 83,
        "Sp. Atk": 100,
        "Sp. Def": 100,
        "Speed": 80
    },
    {
        "#": 3,
        "Name": "Venusaur",
        "Type1": "Grass",
        "Type2": "Poison",
        "Variant": "Mega Venusaur",
        "Total": 625,
        "HP": 80,
        "Attack": 100,
        "De

Salvataggio

In [69]:
# JSON
with open('./Scraping_datasets/pokedex.json', 'w', encoding='utf8') as outfile:
    outfile.write(pokedex_json)
# CSV
pokedex_df.to_csv('./Scraping_datasets/pokedex.csv', na_rep="NA")

In [25]:
#pokedex_json = pokedex_df.to_json(orient='index', indent = 4)
#print(pokedex_json)

##  Items (Name, Category, Effect)

In [56]:
items_URL = "https://pokemondb.net/item/all"

In [58]:
items_page = requests.get(items_URL)

In [59]:
items_soup = BeautifulSoup(items_page.content, parser = 'html.parser')

In [60]:
items_df = pd.read_html(str(items_soup))[0]

In [61]:
items_df

,Name,Category,Effect
0,Ability Capsule,Hold items,A capsule that allows a Pokémon with two Abili...
1,Ability Patch,General items,Changes a Pokémon's ability to its Hidden Abil...
2,Ability Urge,Battle items,"When used, it activates the Ability of an ally..."
3,Abomasite,Hold items,Enables Abomasnow to Mega Evolve during battle.
4,Absolite,Hold items,Enables Absol to Mega Evolve during battle.
...,...,...,...
955,Yellow Scarf,General items,Raises holder's Tough aspect in a Contest.
956,Yellow Shard,General items,A small yellow shard. It appears to be from so...
957,Zap Plate,Hold items,Increases power of Electric-type moves. Change...
958,Zinc,Medicine,Increases Special Defense EVs by 10.


In [74]:
items_json = json.dumps(list(items_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)

In [75]:
print(items_json)

[
    {
        "Name": "Ability Capsule",
        "Category": "Hold items",
        "Effect": "A capsule that allows a Pokémon with two Abilities to switch between these Abilities when it is used."
    },
    {
        "Name": "Ability Patch",
        "Category": "General items",
        "Effect": "Changes a Pokémon's ability to its Hidden Ability."
    },
    {
        "Name": "Ability Urge",
        "Category": "Battle items",
        "Effect": "When used, it activates the Ability of an ally Pokémon."
    },
    {
        "Name": "Abomasite",
        "Category": "Hold items",
        "Effect": "Enables Abomasnow to Mega Evolve during battle."
    },
    {
        "Name": "Absolite",
        "Category": "Hold items",
        "Effect": "Enables Absol to Mega Evolve during battle."
    },
    {
        "Name": "Absorb Bulb",
        "Category": "Hold items",
        "Effect": "A consumable bulb. If the holder is hit by a Water-type move, its Sp. Atk will rise."
    },
    {
        "Na

Salvataggio

In [77]:
# JSON
with open('./Scraping_datasets/items.json', 'w', encoding='utf8') as outfile:
    outfile.write(items_json)
# CSV
items_df.to_csv('./Scraping_datasets/items.csv', na_rep="NA")

## abilities (name, n_pokemon, description, Gen)

In [83]:
abilities_URL = "https://pokemondb.net/ability"
abilities_page = requests.get(abilities_URL)
abilities_soup = BeautifulSoup(abilities_page.content, parser = 'html.parser')
abilities_df = pd.read_html(str(abilities_soup))[0]
abilities_json = json.dumps(list(abilities_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/abilities.json', 'w', encoding='utf8') as outfile:
    outfile.write(abilities_json)
# CSV
abilities_df.to_csv('./Scraping_datasets/abilities.csv', na_rep="NA")

In [79]:
abilities_df

,Name,Pokémon,Description,Gen.
0,Adaptability,16,Powers up moves of the same type.,4
1,Aerilate,2,Turns Normal-type moves into Flying-type moves.,6
2,Aftermath,10,Damages the attacker landing the finishing hit.,4
3,Air Lock,1,Eliminates the effects of weather.,3
4,Analytic,12,Boosts move power when the Pokémon moves last.,5
...,...,...,...,...
261,White Smoke,4,Prevents other Pokémon from lowering its stats.,3
262,Wimp Out,1,Switches out when HP drops below half.,7
263,Wonder Guard,1,Only supereffective moves will hit.,3
264,Wonder Skin,5,Makes status-changing moves more likely to miss.,5


In [82]:
print(abilities_json)

[
    {
        "Name": "Adaptability",
        "Pokémon": 16,
        "Description": "Powers up moves of the same type.",
        "Gen.": 4
    },
    {
        "Name": "Aerilate",
        "Pokémon": 2,
        "Description": "Turns Normal-type moves into Flying-type moves.",
        "Gen.": 6
    },
    {
        "Name": "Aftermath",
        "Pokémon": 10,
        "Description": "Damages the attacker landing the finishing hit.",
        "Gen.": 4
    },
    {
        "Name": "Air Lock",
        "Pokémon": 1,
        "Description": "Eliminates the effects of weather.",
        "Gen.": 3
    },
    {
        "Name": "Analytic",
        "Pokémon": 12,
        "Description": "Boosts move power when the Pokémon moves last.",
        "Gen.": 5
    },
    {
        "Name": "Anger Point",
        "Pokémon": 9,
        "Description": "Maxes Attack after taking a critical hit.",
        "Gen.": 4
    },
    {
        "Name": "Anticipation",
        "Pokémon": 14,
        "Description": "Senses

## Pokémon EVs (Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed)

In [84]:
EV_URL = "https://pokemondb.net/ev/all"
EV_page = requests.get(EV_URL)
EV_soup = BeautifulSoup(EV_page.content, parser = 'html.parser')
EV_df = pd.read_html(str(EV_soup))[0]
EV_json = json.dumps(list(EV_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/EV.json', 'w', encoding='utf8') as outfile:
    outfile.write(EV_json)
# CSV
EV_df.to_csv('./Scraping_datasets/EV.csv', na_rep="NA")

In [85]:
EV_df

,#,Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,NaN,NaN,NaN,1.0,NaN,NaN
1,2,Ivysaur,NaN,NaN,NaN,1.0,1.0,NaN
2,3,Venusaur,NaN,NaN,NaN,2.0,1.0,NaN
3,3,Venusaur Mega Venusaur,NaN,NaN,NaN,2.0,1.0,NaN
4,4,Charmander,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
1071,903,Sneasler,NaN,NaN,NaN,NaN,NaN,NaN
1072,904,Overqwil,NaN,NaN,NaN,NaN,NaN,NaN
1073,905,Enamorus Incarnate Forme,NaN,NaN,NaN,NaN,NaN,NaN
1074,905,Enamorus Therian Forme,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
print(EV_json)

[
    {
        "#": 1,
        "Name": "Bulbasaur",
        "Sp. Atk": 1.0
    },
    {
        "#": 2,
        "Name": "Ivysaur",
        "Sp. Atk": 1.0,
        "Sp. Def": 1.0
    },
    {
        "#": 3,
        "Name": "Venusaur",
        "Sp. Atk": 2.0,
        "Sp. Def": 1.0
    },
    {
        "#": 3,
        "Name": "Venusaur Mega Venusaur",
        "Sp. Atk": 2.0,
        "Sp. Def": 1.0
    },
    {
        "#": 4,
        "Name": "Charmander",
        "Speed": 1.0
    },
    {
        "#": 5,
        "Name": "Charmeleon",
        "Sp. Atk": 1.0,
        "Speed": 1.0
    },
    {
        "#": 6,
        "Name": "Charizard",
        "Sp. Atk": 3.0
    },
    {
        "#": 6,
        "Name": "Charizard Mega Charizard X",
        "Sp. Atk": 3.0
    },
    {
        "#": 6,
        "Name": "Charizard Mega Charizard Y",
        "Sp. Atk": 3.0
    },
    {
        "#": 7,
        "Name": "Squirtle",
        "Defense": 1.0
    },
    {
        "#": 8,
        "Name": "Wartortle",


## Moves

Name
Type
Cat.
Power
Acc.
PP
TM
Effect
Prob. (%))

In [87]:
move_URL = "https://pokemondb.net/move/all"
move_page = requests.get(move_URL)
move_soup = BeautifulSoup(move_page.content, parser = 'html.parser')
move_df = pd.read_html(str(move_soup))[0]
move_json = json.dumps(list(move_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/moves.json', 'w', encoding='utf8') as outfile:
    outfile.write(move_json)
# CSV
move_df.to_csv('./Scraping_datasets/moves.csv', na_rep="NA")

In [88]:
move_df

,Name,Type,Cat.,Power,Acc.,PP,TM,Effect,Prob. (%)
0,"10,000,000 Volt Thunderbolt",Electric,NaN,195,—,1,NaN,Pikachu-exclusive Z-Move. High critical hit ra...,—
1,Absorb,Grass,NaN,20,100,25,NaN,User recovers half the HP inflicted on opponent.,—
2,Accelerock,Rock,NaN,40,100,20,NaN,User attacks first.,—
3,Acid,Poison,NaN,40,100,30,NaN,May lower opponent's Special Defense.,10
4,Acid Armor,Poison,NaN,—,—,20,NaN,Sharply raises user's Defense.,—
...,...,...,...,...,...,...,...,...,...
860,Yawn,Normal,NaN,—,—,10,NaN,Puts opponent to sleep in the next turn.,—
861,Zap Cannon,Electric,NaN,120,50,5,NaN,Paralyzes opponent.,100
862,Zen Headbutt,Psychic,NaN,80,90,15,NaN,May cause flinching.,20
863,Zing Zap,Electric,NaN,80,100,10,NaN,May cause flinching.,30


In [91]:
print(move_json)

[
    {
        "Name": "10,000,000 Volt Thunderbolt",
        "Type": "Electric",
        "Power": "195",
        "Acc.": "—",
        "PP": "1",
        "Effect": "Pikachu-exclusive Z-Move. High critical hit ratio.",
        "Prob. (%)": "—"
    },
    {
        "Name": "Absorb",
        "Type": "Grass",
        "Power": "20",
        "Acc.": "100",
        "PP": "25",
        "Effect": "User recovers half the HP inflicted on opponent.",
        "Prob. (%)": "—"
    },
    {
        "Name": "Accelerock",
        "Type": "Rock",
        "Power": "40",
        "Acc.": "100",
        "PP": "20",
        "Effect": "User attacks first.",
        "Prob. (%)": "—"
    },
    {
        "Name": "Acid",
        "Type": "Poison",
        "Power": "40",
        "Acc.": "100",
        "PP": "30",
        "Effect": "May lower opponent's Special Defense.",
        "Prob. (%)": "10"
    },
    {
        "Name": "Acid Armor",
        "Type": "Poison",
        "Power": "—",
        "Acc.": "—",
      

## Natures and berries (nature_name, Increases, Decreases, Berrie_like, berrie_dislike)

In [100]:
natures_URL = "https://pokemondb.net/mechanics/natures"
natures_page = requests.get(natures_URL)
natures_soup = BeautifulSoup(natures_page.content, parser = 'html.parser')
natures_df = pd.read_html(str(natures_soup))[1]
natures_df

,Nature,Increases,Decreases
0,Adamant,Attack,Sp. Atk
1,Bashful,Sp. Atk,Sp. Atk
2,Bold,Defense,Attack
3,Brave,Attack,Speed
4,Calm,Sp. Def,Attack
5,Careful,Sp. Def,Sp. Atk
6,Docile,Defense,Defense
7,Gentle,Sp. Def,Defense
8,Hardy,Attack,Attack
9,Hasty,Speed,Defense


In base alla natura si hanno bacche che il pokemon apprezza disprezzza

In [101]:
def berries_taste(x):
    if x == 'Attack':
        val = 'Spicy'
    elif x == 'Defense':
        val = 'Sour'
    elif x == 'Speed':
        val = 'Sweet'
    elif x == 'Sp. Atk':
        val = 'Dry'
    elif x == 'Sp. Def': 
        val = 'Bitter'
    return val

In [102]:
natures_df['Likes_berrie'] = natures_df['Increases'].apply(berries_taste)
natures_df['Dislikes_berrie'] = natures_df['Decreases'].apply(berries_taste)

In [103]:
natures_df

,Nature,Increases,Decreases,Likes_berrie,Dislikes_berrie
0,Adamant,Attack,Sp. Atk,Spicy,Dry
1,Bashful,Sp. Atk,Sp. Atk,Dry,Dry
2,Bold,Defense,Attack,Sour,Spicy
3,Brave,Attack,Speed,Spicy,Sweet
4,Calm,Sp. Def,Attack,Bitter,Spicy
5,Careful,Sp. Def,Sp. Atk,Bitter,Dry
6,Docile,Defense,Defense,Sour,Sour
7,Gentle,Sp. Def,Defense,Bitter,Sour
8,Hardy,Attack,Attack,Spicy,Spicy
9,Hasty,Speed,Defense,Sweet,Sour


In [104]:
natures_json = json.dumps(list(natures_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/natures.json', 'w', encoding='utf8') as outfile:
    outfile.write(natures_json)
# CSV
natures_df.to_csv('./Scraping_datasets/natures.csv', na_rep="NA")

# 2. `Pikalytics` Series 12 VGC 2022 (Apr 22)

## Selenium

- *Lista dei pokemon registrati e link alle specifiche pagine*

In [197]:
driver = webdriver.Edge('msedgedriver')

In [198]:
driver.get('https://pikalytics.com/pokedex/ss/')

In [199]:
driver.title

'VGC 2022 Series 12 Stats Pokédex | Pokémon | Pikalytics'

Seleziono la porzione di scermo di interesse

In [202]:
pokemon_list = driver.find_element(by = 'xpath', value = '//*[@id="pokedex_wrapper"]')

Scrollo in modo tale da mostrare tutti i pokemon nell'HTML

In [203]:
from selenium.webdriver.common.action_chains import ActionChains

Muovo il cursore sulla lista (non serve, funziona anche il tab da solo)

In [204]:
actions = ActionChains(driver)
actions.move_to_element(pokemon_list).perform()

Faccio tabbare un numero molto superiore al numero di pokemon in modo da assicurarmi che vengano mostrati tutti

In [205]:
for i in range(1,10000):
    actions.key_down(value = '\ue004').perform()

Estraggo il HTMLResponse Object dalla pagina ora completamente caricata

In [99]:
pikalytics_page = driver.page_source

In [100]:
driver.close()
driver.quit()

## bs4

- *Estraggo la lista dal HTTPResponse Object sottenuto con selenium*

In [102]:
pikalytics_soup = BeautifulSoup(pikalytics_page, features='html.parser')

In [206]:
print(pikalytics_soup.prettify())

<html>
 <head>
  <meta content="initial-scale=1.0; minimum-scale=1.0; maximum-scale=1.0;" name="viewport"/>
  <meta content="app-id=1511370166" name="apple-itunes-app"/>
  <link href="https://fonts.googleapis.com/css?family=Lato:400,700,300,100&amp;display=swap" rel="stylesheet" type="text/css"/>
  <link href="https://cdn.pikalytics.com/styles/styles.285597f9fe320c85.css" rel="stylesheet" type="text/css"/>
  <script async="" src="https://www.google-analytics.com/analytics.js">
  </script>
  <script>
   if (
            (localStorage.getItem('dark-mode') == null &&
                window.matchMedia &&
                window.matchMedia('(prefers-color-scheme: dark)').matches) ||
            localStorage.getItem('dark-mode') == '1'
        ) {
            // Get HTML head element
            var head = document.getElementsByTagName('HEAD')[0]
            // Create new link Element
            var link = document.createElement('link')
            // set the attributes for link element
    

### Lista dei link alle pagine dei singoli pokemon

In [108]:
pikalytics_pkm_min_list_ul = pikalytics_soup.find(id = "min_list")

In [212]:
pikalytics_pkm_min_list_a = pikalytics_pkm_min_list_ul.find_all('a', recursive=False)

In [214]:
pikalytics_pkm_min_list_links = []

In [215]:
for i in pikalytics_pkm_min_list_a:
    pikalytics_pkm_min_list_links.append(i['href'])

In [216]:
pikalytics_pkm_min_list_links

['/pokedex/ss/Zacian-Crowned?l=en',
 '/pokedex/ss/Incineroar?l=en',
 '/pokedex/ss/Kyogre?l=en',
 '/pokedex/ss/Grimmsnarl?l=en',
 '/pokedex/ss/Regieleki?l=en',
 '/pokedex/ss/Rillaboom?l=en',
 '/pokedex/ss/Groudon?l=en',
 '/pokedex/ss/Calyrex-Shadow',
 '/pokedex/ss/Thundurus?l=en',
 '/pokedex/ss/Landorus-Therian',
 '/pokedex/ss/Gastrodon?l=en',
 '/pokedex/ss/Charizard?l=en',
 '/pokedex/ss/Amoonguss?l=en',
 '/pokedex/ss/Indeedee-F',
 '/pokedex/ss/Whimsicott?l=en',
 '/pokedex/ss/Yveltal?l=en',
 '/pokedex/ss/Calyrex-Ice',
 '/pokedex/ss/Palkia?l=en',
 '/pokedex/ss/Porygon2?l=en',
 '/pokedex/ss/Tornadus?l=en',
 '/pokedex/ss/Zapdos?l=en',
 '/pokedex/ss/Kartana?l=en',
 '/pokedex/ss/Blastoise?l=en',
 '/pokedex/ss/Venusaur?l=en',
 '/pokedex/ss/Ferrothorn?l=en',
 '/pokedex/ss/Ditto?l=en',
 '/pokedex/ss/Urshifu?l=en',
 '/pokedex/ss/Xerneas?l=en',
 '/pokedex/ss/Dialga?l=en',
 '/pokedex/ss/Shedinja?l=en',
 '/pokedex/ss/Solgaleo?l=en',
 '/pokedex/ss/Dusclops?l=en',
 '/pokedex/ss/Torkoal?l=en',
 '/poke

### Definizione Dataset Vuoti

In [430]:
usage_S12VGC_columns = ['Pokemon', 'Monthly Usage (k)', 'Usage Percent (%)', 'Monthly Rank']
usage_S12VGC = pd.DataFrame(columns = usage_S12VGC_columns)

basestats_S12VGC_columns = ['Pokemon', 'HP', 'Atk', 'Def', 'SpA', 'SpD', 'Spe']
basestats_S12VGC = pd.DataFrame(columns = basestats_S12VGC_columns)

moves_S12VGC_columns = ['Pokemon', 'Move', 'Type', 'Use_Percentage (%)']
moves_S12VGC = pd.DataFrame(columns = moves_S12VGC_columns)
                            
teammates_S12VGC_columns = ['Pokemon', 'Teammate', 'Use_Percentage (%)']
teammates_S12VGC = pd.DataFrame(columns = teammates_S12VGC_columns)
                            
items_S12VGC_columns = ['Pokemon', 'Item', 'Use_Percentage (%)']
items_S12VGC = pd.DataFrame(columns = items_S12VGC_columns)
                            
abilities_S12VGC_columns = ['Pokemon', 'Ability', 'Use_Percentage (%)']
abilities_S12VGC = pd.DataFrame(columns = abilities_S12VGC_columns)
                            
EV_spread_S12VGC_columns = ['Pokemon', 'Nature', 'HP/Atk/Def/SpA/SpD/Spe', 'Use_Percentage (%)']
EV_spread_S12VGC = pd.DataFrame(columns = EV_spread_S12VGC_columns)

### Scraping bs4

In [431]:
from IPython.display import clear_output
lista_dei_vuoti = []
for i, link in enumerate(pikalytics_pkm_min_list_links):
    
    #print(link)
    
    # Acquisizione soup pagina specifica
    
    pikalytics_pkm_URL = "https://pikalytics.com" + link
    pikalytics_pkm_page = requests.get(pikalytics_pkm_URL)
    pikalytics_pkm_soup = BeautifulSoup(pikalytics_pkm_page.content,
                                        features='html.parser')
    
    ## Controllo che la pagina esista verificando che compaia il nome del pokemon
    if (pikalytics_pkm_soup.find('h1') == None):
        lista_dei_vuoti.append(link)
        continue
    
    pikalytics_pkm_name = (pikalytics_pkm_soup.find('h1').text)

    # Avanzamento
    print(
        f'Avanzamento: {i}/{len(pikalytics_pkm_min_list_links)} - Pokemon: {pikalytics_pkm_name}'
    )

    # Usage Stats

    pikalytics_pkm_usagestats_temp = [f'{pikalytics_pkm_name}']
    pikalytics_pkm_usagestats_div = pikalytics_pkm_soup.find(
        'div', class_="header-div-right header-div-right-margin").find_all(
            'div', class_='pokemon-ind-summary-text')
    ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
    if (len(pikalytics_pkm_usagestats_div) < 3):
        pikalytics_pkm_usagestats_temp = [f'{pikalytics_pkm_name}', '-', '-', '-']
    else:
        for e, stat in enumerate(pikalytics_pkm_usagestats_div):
            if (e == 2):
                pikalytics_pkm_usagestats_temp.append(stat.contents[1])
            else:
                pikalytics_pkm_usagestats_temp.append(stat.contents[0])
    ## Trasformo riga in serie e appendo
    usagestats_series = pd.Series(pikalytics_pkm_usagestats_temp,
                                  index=usage_S12VGC.columns)
    usage_S12VGC = usage_S12VGC.append(usagestats_series, ignore_index=True)

    # Base Stats

    pikalytics_pkm_basestats_temp = [f'{pikalytics_pkm_name}']
    pikalytics_pkm_basestats_stat_wrapper = pikalytics_pkm_soup.find(
        'div', id='bstats_wrapper').find('div',
                                         recursive=False).find('div',
                                                               recursive=False)
    ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
    if pikalytics_pkm_basestats_stat_wrapper:
        pikalytics_pkm_basestats_stat_div = pikalytics_pkm_basestats_stat_wrapper.find_all(
            'div', recursive=False)
        for stat in pikalytics_pkm_basestats_stat_div:
            pikalytics_pkm_basestats_temp.append(
                stat.find_all('div', recursive=False)[1].text)
        ## Trasformo riga in serie e appendo
        base_stat_series = pd.Series(pikalytics_pkm_basestats_temp,
                                     index=basestats_S12VGC.columns)
        basestat_S12VGC = basestat_S12VGC.append(base_stat_series,
                                                 ignore_index=True)
    # Moves Stats

    pikalytics_pkm_moves_wrapper = pikalytics_pkm_soup.find(id="moves_wrapper")
    pikalytics_pkm_moves_div = pikalytics_pkm_moves_wrapper.find_all(
        'div', class_="pokedex-move-entry-new")
    for moves in pikalytics_pkm_moves_div:
        # In questo caso, essendoci più mosse consigliate per ogni pokemon, dobbiamo creare una lista per ogni mossa (una riga nel dataframe)
        pikalytics_pkm_moves_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not moves.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 3 variabili
        move = moves.find_all('div', recursive=False)
        pikalytics_pkm_moves_temp.append(move[0].text)
        pikalytics_pkm_moves_temp.append(move[1].find('span').text)
        pikalytics_pkm_moves_temp.append(move[2].text)
        ## Trasformo riga in serie e appendo
        moves_series = pd.Series(pikalytics_pkm_moves_temp,
                                 index=moves_S12VGC.columns)
        moves_S12VGC = moves_S12VGC.append(moves_series, ignore_index=True)

    # Teammates Stats

    pikalytics_pkm_teammates_wrapper = pikalytics_pkm_soup.find(
        id="dex_team_wrapper")
    pikalytics_pkm_teammates_div = pikalytics_pkm_teammates_wrapper.find_all(
        'a', class_="teammate_entry", recursive=False)
    for teammates in pikalytics_pkm_teammates_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_teammates_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not teammates.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 2 variabili
        pikalytics_pkm_teammates_temp.append(teammates['data-name'])
        pikalytics_pkm_teammates_temp.append(
            teammates.find_all('div', recursive=False)[3].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        teammates_series = pd.Series(pikalytics_pkm_teammates_temp,
                                     index=teammates_S12VGC.columns)
        teammates_S12VGC = teammates_S12VGC.append(teammates_series,
                                                   ignore_index=True)

    # Items Stats

    pikalytics_pkm_items_wrapper = pikalytics_pkm_soup.find(
        id="items_wrapper").find('div')
    pikalytics_pkm_items_div = pikalytics_pkm_items_wrapper.find_all(
        'div', recursive=False)
    for item in pikalytics_pkm_items_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_items_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not item.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 2 variabili
        pikalytics_pkm_items_temp.append(
            item.find_all('div', recursive=False)[1].text)
        pikalytics_pkm_items_temp.append(
            item.find_all('div', recursive=False)[2].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        items_series = pd.Series(pikalytics_pkm_items_temp,
                                 index=items_S12VGC.columns)
        items_S12VGC = items_S12VGC.append(items_series, ignore_index=True)
    
    # Abilities

    pikalytics_pkm_abilities_wrapper = pikalytics_pkm_soup.find(
        id="abilities_wrapper").find('div')
    pikalytics_pkm_abilities_div = pikalytics_pkm_abilities_wrapper.find_all(
        'div', recursive=False)
    for ability in pikalytics_pkm_abilities_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_abilities_temp = [f'{pikalytics_pkm_name}']
        # Vado ad aggiungere separatamente le 2 variabili
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not ability.find_all('div', recursive=False):
            break
        pikalytics_pkm_abilities_temp.append(
            ability.find_all('div', recursive=False)[0].text)
        pikalytics_pkm_abilities_temp.append(
            ability.find_all('div', recursive=False)[1].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        abilities_series = pd.Series(pikalytics_pkm_abilities_temp,
                                 index=abilities_S12VGC.columns)
        abilities_S12VGC = abilities_S12VGC.append(abilities_series, ignore_index=True)
        
    # ev_spread

    pikalytics_pkm_ev_spread_wrapper = pikalytics_pkm_soup.find(
        id="dex_spreads_wrapper").find('div')
    pikalytics_pkm_ev_spread_div = pikalytics_pkm_ev_spread_wrapper.find_all(
        'div', recursive=False)
    for ev_spread in pikalytics_pkm_ev_spread_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_ev_spread_temp = [f'{pikalytics_pkm_name}']
        # Vado ad aggiungere separatamente le 3 variabili
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not ev_spread.find_all('div', recursive=False):
            break
        ## Natura
        pikalytics_pkm_ev_spread_temp.append(
            ev_spread.find_all('div', recursive=False)[0].text)
        ## Spreads li vado a sommare per ottenere un'unica stat
        spread_div = ev_spread.find_all('div', recursive=False)[1:7]
        spread_temp = ""
        for ev in (ev_spread.find_all('div', recursive=False)[1:7]):
            spread_temp += ev.text
        pikalytics_pkm_ev_spread_temp.append(spread_temp)
        ## Percentage
        pikalytics_pkm_ev_spread_temp.append(
            ev_spread.find_all('div', recursive=False)[7].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_ev_spread_temp)
        ev_spread_series = pd.Series(pikalytics_pkm_ev_spread_temp,
                                 index=EV_spread_S12VGC.columns)
        EV_spread_S12VGC = EV_spread_S12VGC.append(ev_spread_series, ignore_index=True)
    clear_output(wait=True)
    #if (i > 5):
    #    break

Avanzamento: 634/662 - Pokemon: Gourgeist-Large


## Dataset Ottenuti

In [432]:
lista_dei_vuoti

['/pokedex/ss/Avalugg?l=en',
 '/pokedex/ss/Regirock?l=en',
 '/pokedex/ss/Lanturn?l=en',
 '/pokedex/ss/Doublade?l=en',
 '/pokedex/ss/Piloswine?l=en',
 '/pokedex/ss/Zacian?l=en',
 '/pokedex/ss/Rhydon?l=en',
 '/pokedex/ss/Gothorita?l=en',
 '/pokedex/ss/Regice?l=en',
 '/pokedex/ss/Zamazenta?l=en',
 '/pokedex/ss/Azelf?l=en',
 '/pokedex/ss/Spiritomb?l=en',
 '/pokedex/ss/Ferroseed?l=en',
 '/pokedex/ss/Cramorant?l=en',
 '/pokedex/ss/Armaldo?l=en',
 '/pokedex/ss/Slowking-Galar',
 '/pokedex/ss/Cryogonal?l=en',
 '/pokedex/ss/Morgrem?l=en',
 '/pokedex/ss/Carbink?l=en',
 '/pokedex/ss/Frosmoth?l=en',
 '/pokedex/ss/Relicanth?l=en',
 '/pokedex/ss/Jynx?l=en',
 '/pokedex/ss/Barbaracle?l=en',
 '/pokedex/ss/Latios?l=en',
 '/pokedex/ss/Quagsire?l=en',
 '/pokedex/ss/Naganadel?l=en',
 '/pokedex/ss/Aurorus?l=en',
 '/pokedex/ss/Guzzlord?l=en',
 '/pokedex/ss/Bronzor?l=en',
 '/pokedex/ss/Herdier?l=en',
 '/pokedex/ss/Feebas?l=en',
 '/pokedex/ss/Archeops?l=en',
 '/pokedex/ss/Silvally-Ground',
 '/pokedex/ss/Yamper?

In [433]:
display(usage_S12VGC)

,Pokemon,Monthly Usage (k),Usage Percent (%),Monthly Rank
0,Zacian-Crowned,1088,65,1
1,Incineroar,1124,59,2
2,Kyogre,744,35,3
3,Grimmsnarl,577,30,4
4,Regieleki,603,29,5
...,...,...,...,...
337,Steelix,-,-,-
338,Noctowl,-,-,-
339,Corsola,-,-,-
340,Mime Jr.,-,-,-


In [434]:
display(basestat_S12VGC)

,Pokemon,HP,Atk,Def,SpA,SpD,Spe
0,Zacian-Crowned,92,170,115,80,115,148
1,Grimmsnarl,95,120,65,95,75,60
2,Regieleki,80,100,50,100,50,200
3,Rillaboom,100,125,90,60,70,85
4,Calyrex-Shadow,100,85,80,165,100,150
...,...,...,...,...,...,...,...
1420,Steelix,75,85,200,55,65,30
1421,Noctowl,100,50,50,86,96,70
1422,Corsola,65,55,95,65,95,35
1423,Mime Jr.,20,25,45,70,90,60


In [435]:
display(moves_S12VGC)

,Pokemon,Move,Type,Use_Percentage (%)
0,Zacian-Crowned,Behemoth Blade,steel,99.996%
1,Zacian-Crowned,Protect,normal,98.672%
2,Zacian-Crowned,Sacred Sword,fighting,84.154%
3,Zacian-Crowned,Play Rough,fairy,64.487%
4,Zacian-Crowned,Substitute,normal,30.274%
...,...,...,...,...
3613,Mime Jr.,Other,,12.562%
3614,Gourgeist-Large,Power Whip,grass,100.000%
3615,Gourgeist-Large,Trick Room,psychic,99.986%
3616,Gourgeist-Large,Imprison,psychic,99.986%


In [436]:
display(teammates_S12VGC)

,Pokemon,Teammate,Use_Percentage (%)
0,Zacian-Crowned,Incineroar,54.189%
1,Zacian-Crowned,Kyogre,31.467%
2,Zacian-Crowned,Grimmsnarl,31.148%
3,Zacian-Crowned,Rillaboom,29.083%
4,Zacian-Crowned,Gastrodon,24.347%
...,...,...,...
3417,Gourgeist-Large,Weezing-Galar,0.014%
3418,Gourgeist-Large,Duraludon,0.014%
3419,Gourgeist-Large,Togetic,0.014%
3420,Gourgeist-Large,Talonflame,0.014%


In [437]:
display(items_S12VGC)

,Pokemon,Item,Use_Percentage (%)
0,Zacian-Crowned,Rusted Sword,100.000%
1,Incineroar,Shuca Berry,36.082%
2,Incineroar,Safety Goggles,23.390%
3,Incineroar,Sitrus Berry,14.031%
4,Incineroar,Focus Sash,8.994%
...,...,...,...
1887,Corsola,Other,0.734%
1888,Mime Jr.,Focus Sash,99.860%
1889,Mime Jr.,Other,0.140%
1890,Gourgeist-Large,Figy Berry,99.986%


In [438]:
display(abilities_S12VGC)

,Pokemon,Ability,Use_Percentage (%)
0,Zacian-Crowned,Intrepid Sword,100.000%
1,Incineroar,Intimidate,99.927%
2,Incineroar,Blaze,0.073%
3,Kyogre,Drizzle,100.000%
4,Grimmsnarl,Prankster,99.959%
...,...,...,...
768,Mime Jr.,Soundproof,99.870%
769,Mime Jr.,Filter,0.130%
770,Mime Jr.,Technician,0.000%
771,Gourgeist-Large,Frisk,100.000%


In [439]:
display(EV_spread_S12VGC)

,Pokemon,Nature,HP/Atk/Def/SpA/SpD/Spe,Use_Percentage (%)
0,Zacian-Crowned,Jolly,0/252/0/0/4/252,10.388%
1,Zacian-Crowned,Adamant,252/28/4/0/12/212,4.504%
2,Zacian-Crowned,Jolly,4/252/0/0/0/252,4.419%
3,Zacian-Crowned,Adamant,188/164/4/0/4/148,2.891%
4,Zacian-Crowned,Jolly,0/252/4/0/0/252,2.456%
...,...,...,...,...
6814,Gourgeist-Large,Sassy,252/4/0/0/252/0,0.000%
6815,Gourgeist-Large,Relaxed,252/0/252/4/0/0,0.000%
6816,Gourgeist-Large,Quiet,252/0/0/252/4/0,0.000%
6817,Gourgeist-Large,Adamant,252/60/0/0/196/0,0.000%


## Salvataggio

In [440]:
usage_S12VGC.to_csv('./Scraping_datasets/pikalytics/usage_S12VGC.csv', na_rep="NA")
basestat_S12VGC.to_csv('./Scraping_datasets/pikalytics/basestat_S12VGC.csv', na_rep="NA")
moves_S12VGC.to_csv('./Scraping_datasets/pikalytics/moves_S12VGC.csv', na_rep="NA")
teammates_S12VGC.to_csv('./Scraping_datasets/pikalytics/teammates_S12VGC.csv', na_rep="NA")
items_S12VGC.to_csv('./Scraping_datasets/pikalytics/items_S12VGC.csv', na_rep="NA")
abilities_S12VGC.to_csv('./Scraping_datasets/pikalytics/abilities_S12VGC.csv', na_rep="NA")
EV_spread_S12VGC.to_csv('./Scraping_datasets/pikalytics/EV_spread_S12VGC.csv', na_rep="NA")